# Recommender Systems - Алгоритм AlgoSom

Рекомендательная система с использованием:
- Collaborative Filtering (ALS)
- Content-Based Filtering
- Hybrid подход
- Matrix Factorization
- Implicit feedback

In [ ]:
!pip install implicit pandas numpy scikit-learn scipy surprise lightfm -q

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
import implicit
from surprise import Dataset, Reader, SVD, NMF
from surprise.model_selection import cross_validate
import warnings
warnings.filterwarnings('ignore')

print("✓ Библиотеки загружены!")

## 1. Загрузка данных

In [ ]:
# === ВАШИ ДАННЫЕ ===
# Формат: user_id, item_id, rating (или interaction)
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Названия колонок
USER_COL = 'user_id'
ITEM_COL = 'item_id'
RATING_COL = 'rating'  # или 'interaction', 'clicks', etc.

print(f"Train interactions: {len(train_df)}")
print(f"Test interactions: {len(test_df)}")
print(f"\nУникальных пользователей: {train_df[USER_COL].nunique()}")
print(f"Уникальных айтемов: {train_df[ITEM_COL].nunique()}")
print(f"\nПример данных:")
print(train_df.head())
print(f"\nСтатистика рейтингов:")
print(train_df[RATING_COL].describe())

## 2. Encoding user и item ID

In [ ]:
# Label encoding для user и item
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

# Fit на всех уникальных значениях (train + test)
all_users = pd.concat([train_df[USER_COL], test_df[USER_COL]]).unique()
all_items = pd.concat([train_df[ITEM_COL], test_df[ITEM_COL]]).unique()

user_encoder.fit(all_users)
item_encoder.fit(all_items)

# Transform
train_df['user_encoded'] = user_encoder.transform(train_df[USER_COL])
train_df['item_encoded'] = item_encoder.transform(train_df[ITEM_COL])
test_df['user_encoded'] = user_encoder.transform(test_df[USER_COL])
test_df['item_encoded'] = item_encoder.transform(test_df[ITEM_COL])

n_users = len(user_encoder.classes_)
n_items = len(item_encoder.classes_)

print(f"✓ Encoding завершен")
print(f"Users: {n_users}, Items: {n_items}")

## 3. Создание User-Item матрицы

In [ ]:
def create_interaction_matrix(df, n_users, n_items):
    """
    Создание разреженной User-Item матрицы
    """
    interactions = coo_matrix(
        (
            df[RATING_COL].values,
            (df['user_encoded'].values, df['item_encoded'].values)
        ),
        shape=(n_users, n_items)
    )
    
    return interactions.tocsr()

# Создание матрицы
train_matrix = create_interaction_matrix(train_df, n_users, n_items)

print(f"✓ User-Item матрица создана: {train_matrix.shape}")
print(f"Разреженность: {(1 - train_matrix.nnz / (n_users * n_items)) * 100:.2f}%")

## 4. Метод 1: ALS (Alternating Least Squares)

In [ ]:
# Implicit ALS модель
als_model = implicit.als.AlternatingLeastSquares(
    factors=100,  # размерность латентных факторов
    regularization=0.01,
    iterations=50,
    calculate_training_loss=True,
    random_state=42
)

print("Обучение ALS модели...")

# ALS работает с item-user матрицей (транспонированной)
als_model.fit(train_matrix.T)

print("✓ ALS модель обучена!")
print(f"User factors shape: {als_model.user_factors.shape}")
print(f"Item factors shape: {als_model.item_factors.shape}")

### 4.1 Рекомендации с ALS

In [ ]:
def get_als_recommendations(user_id, n_items=10):
    """
    Получение топ-N рекомендаций для пользователя
    """
    user_idx = user_encoder.transform([user_id])[0]
    
    # Получаем рекомендации
    ids, scores = als_model.recommend(
        user_idx,
        train_matrix[user_idx],
        N=n_items,
        filter_already_liked_items=True
    )
    
    # Декодируем item IDs
    recommended_items = item_encoder.inverse_transform(ids)
    
    return list(zip(recommended_items, scores))

# Пример рекомендаций
sample_user = train_df[USER_COL].iloc[0]
recommendations = get_als_recommendations(sample_user, n_items=10)

print(f"\nРекомендации для пользователя {sample_user}:")
for item, score in recommendations:
    print(f"  Item: {item}, Score: {score:.4f}")

## 5. Метод 2: BPR (Bayesian Personalized Ranking)

In [ ]:
# BPR оптимизирован для ranking задач
bpr_model = implicit.bpr.BayesianPersonalizedRanking(
    factors=100,
    learning_rate=0.01,
    regularization=0.01,
    iterations=100,
    random_state=42
)

print("Обучение BPR модели...")
bpr_model.fit(train_matrix.T)

print("✓ BPR модель обучена!")

## 6. Метод 3: Item-Item Collaborative Filtering

In [ ]:
# Item similarity с cosine distance
item_similarity_model = implicit.nearest_neighbours.CosineRecommender(
    K=50,  # количество соседей
)

print("Обучение Item-Item CF...")
item_similarity_model.fit(train_matrix.T)

print("✓ Item-Item CF обучена!")

## 7. Метод 4: SVD (Surprise library)

In [ ]:
# Подготовка данных для Surprise
reader = Reader(rating_scale=(train_df[RATING_COL].min(), train_df[RATING_COL].max()))

surprise_data = Dataset.load_from_df(
    train_df[[USER_COL, ITEM_COL, RATING_COL]], 
    reader
)

trainset = surprise_data.build_full_trainset()

# SVD модель
svd_model = SVD(
    n_factors=100,
    n_epochs=50,
    lr_all=0.005,
    reg_all=0.02,
    random_state=42
)

print("Обучение SVD...")
svd_model.fit(trainset)

print("✓ SVD модель обучена!")

## 8. Метод 5: Content-Based (если есть признаки айтемов)

In [ ]:
# Если у вас есть признаки айтемов (жанры, категории, эмбеддинги и т.д.)

# === ВАШИ ПРИЗНАКИ АЙТЕМОВ ===
# item_features = pd.read_csv('item_features.csv')
# Например: item_id, genre1, genre2, category, etc.

# Пример: создание item similarity на основе признаков
# item_feature_matrix = ... # матрица признаков
# item_content_similarity = cosine_similarity(item_feature_matrix)

print("Content-based подход требует признаков айтемов")
print("Добавьте item_features.csv если они есть")

## 9. Ансамбль моделей (гибридный подход)

In [ ]:
def hybrid_recommendations(user_id, n_items=10, weights=None):
    """
    Гибридные рекомендации: комбинация ALS, BPR и Item-Item CF
    """
    if weights is None:
        weights = {'als': 0.5, 'bpr': 0.3, 'item_cf': 0.2}
    
    user_idx = user_encoder.transform([user_id])[0]
    user_items = train_matrix[user_idx]
    
    # Получаем scores от каждой модели
    # ALS
    als_ids, als_scores = als_model.recommend(
        user_idx, user_items, N=n_items*3, filter_already_liked_items=True
    )
    als_dict = dict(zip(als_ids, als_scores))
    
    # BPR
    bpr_ids, bpr_scores = bpr_model.recommend(
        user_idx, user_items, N=n_items*3, filter_already_liked_items=True
    )
    bpr_dict = dict(zip(bpr_ids, bpr_scores))
    
    # Item-Item CF
    itemcf_ids, itemcf_scores = item_similarity_model.recommend(
        user_idx, user_items, N=n_items*3, filter_already_liked_items=True
    )
    itemcf_dict = dict(zip(itemcf_ids, itemcf_scores))
    
    # Объединяем scores
    all_items = set(als_dict.keys()) | set(bpr_dict.keys()) | set(itemcf_dict.keys())
    
    hybrid_scores = {}
    for item in all_items:
        score = (
            weights['als'] * als_dict.get(item, 0) +
            weights['bpr'] * bpr_dict.get(item, 0) +
            weights['item_cf'] * itemcf_dict.get(item, 0)
        )
        hybrid_scores[item] = score
    
    # Сортируем и берем топ-N
    sorted_items = sorted(hybrid_scores.items(), key=lambda x: x[1], reverse=True)[:n_items]
    
    # Декодируем
    recommended_items = [(item_encoder.inverse_transform([item_idx])[0], score) 
                         for item_idx, score in sorted_items]
    
    return recommended_items

# Пример гибридных рекомендаций
hybrid_recs = hybrid_recommendations(sample_user, n_items=10)

print(f"\nГибридные рекомендации для пользователя {sample_user}:")
for item, score in hybrid_recs:
    print(f"  Item: {item}, Score: {score:.4f}")

## 10. Популярные айтемы (baseline)

In [ ]:
# Топ популярных айтемов как fallback
popular_items = (
    train_df.groupby(ITEM_COL)[RATING_COL]
    .agg(['count', 'mean'])
    .sort_values('count', ascending=False)
)

print("\nТоп-10 популярных айтемов:")
print(popular_items.head(10))

## 11. Генерация предсказаний для test

In [ ]:
def predict_rating(user_id, item_id, model='als'):
    """
    Предсказание рейтинга для пары user-item
    """
    try:
        user_idx = user_encoder.transform([user_id])[0]
        item_idx = item_encoder.transform([item_id])[0]
    except:
        # Cold start - возвращаем средний рейтинг
        return train_df[RATING_COL].mean()
    
    if model == 'als':
        # Dot product user и item факторов
        rating = als_model.user_factors[user_idx].dot(als_model.item_factors[item_idx])
    elif model == 'bpr':
        rating = bpr_model.user_factors[user_idx].dot(bpr_model.item_factors[item_idx])
    elif model == 'svd':
        rating = svd_model.predict(user_id, item_id).est
    else:
        rating = train_df[RATING_COL].mean()
    
    return rating

# Генерация предсказаний для test
print("\nГенерация предсказаний для test...")

test_predictions = []
for idx, row in test_df.iterrows():
    user_id = row[USER_COL]
    item_id = row[ITEM_COL]
    
    # Используем ALS (или можно ансамбль)
    pred_rating = predict_rating(user_id, item_id, model='als')
    test_predictions.append(pred_rating)

test_df['predicted_rating'] = test_predictions

print(f"✓ Предсказания готовы!")
print(f"\nСтатистика предсказаний:")
print(test_df['predicted_rating'].describe())

## 12. Метрики (если есть ground truth)

In [ ]:
# Если в test есть реальные рейтинги (для валидации)
if RATING_COL in test_df.columns:
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    
    mae = mean_absolute_error(test_df[RATING_COL], test_df['predicted_rating'])
    rmse = mean_squared_error(test_df[RATING_COL], test_df['predicted_rating'], squared=False)
    
    print(f"\nМетрики на test:")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
else:
    print("\nTest не содержит ground truth рейтингов")

## 13. Submission

In [ ]:
submission = pd.DataFrame({
    'user_id': test_df[USER_COL],
    'item_id': test_df[ITEM_COL],
    'predicted_rating': test_df['predicted_rating']
})

submission.to_csv('recsys_submission.csv', index=False)
print("\n✓ Submission сохранен!")
print(submission.head(10))